Youtube tutorial Link: https://www.youtube.com/watch?v=w8yWXqWQYmU  
Author's kaggle notebook: https://www.kaggle.com/code/wwsalmon/simple-mnist-nn-from-scratch-numpy-no-tf-keras

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#getting the dataset using pytorch
from torchvision import datasets, transforms  

# Download MNIST dataset  
mnist_train = datasets.MNIST(root="./data", train=True, download=True)#transform=transforms.ToTensor())  
mnist_test = datasets.MNIST(root="./data", train=False, download=True)#, transform=transforms.ToTensor())  

# Check dataset length  
print(len(mnist_train), len(mnist_test)) 

In [7]:
from tensorflow.keras.datasets import mnist  

# Load MNIST dataset  
(x_train, y_train), (x_test, y_test) = mnist.load_data() 

In [8]:
x_train.flatten().shape

(47040000,)

In [9]:
x_train[0].shape

(28, 28)

In [10]:
x_train = x_train.reshape((60000, 784)).transpose()
x_test = x_test.reshape((10000, 784)).transpose()

In [11]:
#Now each column is a 28x28 image
x_train[:,0].shape

(784,)

In [70]:
def initialzing_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z2):
    return np.exp(Z2) / np.sum(np.exp(Z2))
    
def f_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def b_prop(Z1, A1, Z2, A2, W1, W2, Y):
    m = Y.size
    deriv_ReLU = Z1>0
    one_hot_y = one_hot(Y)
    
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU
    dW1 = 1 / m * dZ1.dot(A1.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha*dW1
    b1 = b1 - alpha*db1
    
    W2 = W2 - alpha*dW2
    b2 = b2 - alpha*db2
    
    return W1, b1, W2, b2 
    

In [75]:
def predict(A2):
    return np.argmax(A2, 0)

def get_accuracy(preds, Y):
    print(preds, Y)
    return np.sum(preds==Y)/ Y.size

In [76]:
def gradient_descent(X, Y, iterations, alpha = 0.01):
    W1, b1, W2, b2 = initialzing_params()
    
    for i in range(iterations):
        Z1, A1, Z2, A2 = f_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = b_prop(Z1, A1, Z2, A2, W1, W2, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        
        if i%10 == 0:
            print(f'Iteration: {i}')
            accuracy = get_accuracy(predict(A2), Y)
            print(f'Accuracy: {accuracy}')
            
    return W1, b1, W2, b2, accuracy

In [77]:
W1, b1, W2, b2, accuracy = gradient_descent(x_train, y_train, 100)

C:\Users\HP 15\AppData\Local\Temp\ipykernel_13832\3962810796.py:12: RuntimeWarning: overflow encountered in exp
  A = np.exp(Z) / sum(np.exp(Z))
C:\Users\HP 15\AppData\Local\Temp\ipykernel_13832\3962810796.py:12: RuntimeWarning: invalid value encountered in true_divide
  A = np.exp(Z) / sum(np.exp(Z))


ValueError: operands could not be broadcast together with shapes (10,784) (10,10) 